In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, callback
from optuna.integration import XGBoostPruningCallback
import mlflow
from utils.metrics import Metrics_df,print_metrics, precision, recall, f1


In [2]:


import pandas as pd
from sklearn.metrics import roc_auc_score
from Config import config
import warnings
warnings.filterwarnings("ignore")
import optuna  # pip install optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc
import pickle
import tempfile
from optuna.integration.mlflow import MLflowCallback


In [3]:
from lightgbm.basic import LightGBMError
from lightgbm import LGBMClassifier, early_stopping, log_evaluation, Booster
from sklearn.metrics import roc_auc_score
import os

# Loading data and group split

In [4]:
# df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data.csv")
# df_pairs = df_pairs.sample(frac=1).reset_index(drop=True)
# kf = GroupKFold(n_splits=5)
# for i, (trn_idx, val_idx) in enumerate(kf.split(df_pairs, df_pairs["placeId1"], df_pairs["placeId1"])):
#     df_pairs.loc[val_idx, "fold"] = i
# df_pairs["fold"].value_counts()
# df_pairs.to_csv("/workspace/Training_data/training/Full_training_data_fold.csv",index=None)

# Loading data

In [5]:
df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data_fold_v4_without_categories.csv")

In [6]:
features = ['similarity', 'haversine', 'name_davies',
       'name_leven', 'name_dice', 'name_jaro', 'name_set_ratio',
       'street_davies', 'street_leven', 'street_jaro', 'email_davies',
       'email_leven', 'email_jaro', 'url_davies', 'url_leven', 'url_jaro',
       'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
       'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
       'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
       'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
       'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
       'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
       'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
       'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
       'Is_name_number_match_1', 'Is_name_number_match_2',
       'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
       'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
       'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0',
       'Is_url_match_1', 'Is_url_match_2']

In [7]:
# from optuna.integration import LightGBMPruningCallback

# def objective(trial, X_train,X_test,Y_train,Y_test):
#     param_grid = {
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5,step=0.1),
#         "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
#         "min_child_samples": trial.suggest_int("min_child_samples", 200, 10000, step=50),
#         "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 1.0,step=0.01),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0,step=0.1),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0,step=0.1),
#         "reg_alpha": trial.suggest_int("reg_alpha", 1, 100, step=1),
#         "reg_lambda": trial.suggest_int("reg_lambda", 1, 100, step=1),
#         "min_split_gain": trial.suggest_float("min_split_gain", 0, 15),
#         "path_smooth": trial.suggest_float("path_smooth", 0.0, 1.0,step=0.1)
#     }
    
#     callbacks = [log_evaluation(100),]
#     callbacks += [early_stopping(100)]
#     callbacks += [LightGBMPruningCallback(trial, "auc")]

        
#     model = LGBMClassifier(objective="binary",n_estimators=10000, **param_grid,n_jobs=-1)
#     model.fit(
#             X_train,
#             Y_train,
#             eval_set=[(X_test, Y_test)],
#             eval_metric="auc",
#             callbacks=callbacks
#         )
#     preds = model.predict_proba(X_test)[:, 1]


#     return roc_auc_score(Y_test, preds)

In [8]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, df_pairs):
    param_grid = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "reg_alpha": trial.suggest_int("reg_alpha", 0, 100, step=5),
        "reg_lambda": trial.suggest_int("reg_lambda", 0, 100, step=5),
        "min_split_gain": trial.suggest_float("min_split_gain", 0, 15),
        "subsample": trial.suggest_float("subsample", 0.2, 0.95, step=0.1),
        "subsample_freq": trial.suggest_categorical("subsample_freq", [1]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 0.95, step=0.1),
        "path_smooth": trial.suggest_float("path_smooth", 0.0, 1.0,step=0.1)
    }
    
    callbacks = [log_evaluation(100),]
    callbacks += [early_stopping(100)]
    callbacks += [LightGBMPruningCallback(trial, "auc")]

    cv_scores = np.empty(5)
    cv_scores_train = np.empty(5)
    for fold in [0,1,2,3,4]:
        X_train = df_pairs[df_pairs["fold"]!=fold]
        X_test = df_pairs[df_pairs["fold"]==fold]
        
        model = LGBMClassifier(objective="binary",n_estimators=10000, **param_grid,n_jobs=-1)
        model.fit(
            X_train[features],
            X_train["duplicate_flag"],
            eval_set=[(X_test[features], X_test["duplicate_flag"])],
            eval_metric="auc",
            callbacks=callbacks
        )
        
        preds = model.predict_proba(X_test[features])[:, 1]
        preds_train = model.predict_proba(X_train[features])[:, 1]
        cv_scores[fold] = roc_auc_score(X_test["duplicate_flag"], preds)
        cv_scores_train[fold] = roc_auc_score(X_train["duplicate_flag"], preds_train)
    print(f"train auc score = {np.mean(cv_scores_train)}")
    print(f"test auc score = {np.mean(cv_scores)}")
    mlflow.log_metric('train_auc',np.mean(cv_scores_train))
    mlflow.log_metric('test_auc',np.mean(cv_scores))
    return np.mean(cv_scores)

In [9]:
date_log = datetime.now().strftime("%Y_%m_%d_%H_%M")
study = optuna.create_study(direction="maximize", study_name=f"parameter_tuning_{config.country}_{date_log}")
mlflow_callback = MLflowCallback(nest_trials =True,tracking_uri=mlflow.get_tracking_uri(),metric_name = 'auc')
func = lambda trial: objective(trial, df_pairs)
study.optimize(func, n_trials=100,callbacks=[mlflow_callback])
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2023-01-13 10:10:05,511] A new study created in memory with name: parameter_tuning_NZL_2023_01_13_10_10


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.999704	valid_0's binary_logloss: 0.0124396
[200]	valid_0's auc: 0.999712	valid_0's binary_logloss: 0.012283
[300]	valid_0's auc: 0.999715	valid_0's binary_logloss: 0.0121891
[400]	valid_0's auc: 0.999717	valid_0's binary_logloss: 0.0121448
[500]	valid_0's auc: 0.99972	valid_0's binary_logloss: 0.012066
Early stopping, best iteration is:
[485]	valid_0's auc: 0.99972	valid_0's binary_logloss: 0.012066
[100]	valid_0's auc: 0.999739	valid_0's binary_logloss: 0.0115226
[200]	valid_0's auc: 0.999746	valid_0's binary_logloss: 0.0113889
[300]	valid_0's auc: 0.999748	valid_0's binary_logloss: 0.0113383
[400]	valid_0's auc: 0.999751	valid_0's binary_logloss: 0.0112854
[500]	valid_0's auc: 0.999752	valid_0's binary_logloss: 0.0112717
[600]	valid_0's auc: 0.999751	valid_0's binary_logloss: 0.0112537
[700]	valid_0's auc: 0.999751	valid_0's binary_logloss: 0.0112382
Early stopping, best iteration is:
[614]	valid_0's

[I 2023-01-13 10:15:04,237] Trial 0 finished with value: 0.9997633156017727 and parameters: {'learning_rate': 0.1521191700157443, 'num_leaves': 1220, 'max_depth': 11, 'min_child_samples': 900, 'max_bin': 164, 'reg_alpha': 40, 'reg_lambda': 75, 'min_split_gain': 3.82491953699631, 'subsample': 0.30000000000000004, 'subsample_freq': 1, 'colsample_bytree': 0.6000000000000001, 'path_smooth': 0.8}. Best is trial 0 with value: 0.9997633156017727.
2023/01/13 10:15:04 INFO mlflow.tracking.fluent: Experiment with name 'parameter_tuning_NZL_2023_01_13_10_10' does not exist. Creating a new experiment.


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.99975	valid_0's binary_logloss: 0.0126265
[200]	valid_0's auc: 0.99976	valid_0's binary_logloss: 0.0121608
[300]	valid_0's auc: 0.99976	valid_0's binary_logloss: 0.0120825
[400]	valid_0's auc: 0.999763	valid_0's binary_logloss: 0.0120355
[500]	valid_0's auc: 0.999763	valid_0's binary_logloss: 0.0120075
Early stopping, best iteration is:
[431]	valid_0's auc: 0.999764	valid_0's binary_logloss: 0.012016
[100]	valid_0's auc: 0.999792	valid_0's binary_logloss: 0.0118814
[200]	valid_0's auc: 0.999803	valid_0's binary_logloss: 0.0114033
[300]	valid_0's auc: 0.999804	valid_0's binary_logloss: 0.0113433
Early stopping, best iteration is:
[293]	valid_0's auc: 0.999804	valid_0's binary_logloss: 0.0113433
[100]	valid_0's auc: 0.999795	valid_0's binary_logloss: 0.0124035
[200]	valid_0's auc: 0.999801	valid_0's binary_logloss: 0.0118964
[300]	valid_0's auc: 0.999804	valid_0's binary_logloss: 0.0117935
[400]	valid_0'

[I 2023-01-13 10:19:04,788] Trial 1 finished with value: 0.9997971727064285 and parameters: {'learning_rate': 0.14005112634484984, 'num_leaves': 1720, 'max_depth': 7, 'min_child_samples': 9500, 'max_bin': 179, 'reg_alpha': 20, 'reg_lambda': 75, 'min_split_gain': 14.526008686053604, 'subsample': 0.5, 'subsample_freq': 1, 'colsample_bytree': 0.4, 'path_smooth': 0.7000000000000001}. Best is trial 1 with value: 0.9997971727064285.


...

Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:02:25,747] Trial 89 pruned. Trial was pruned at iteration 10.
[I 2023-01-13 11:02:30,729] Trial 90 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:35,687] Trial 91 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:41,066] Trial 92 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:45,450] Trial 93 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:50,261] Trial 94 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:54,865] Trial 95 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:02:59,870] Trial 96 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:03:04,371] Trial 97 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:03:08,834] Trial 98 pruned. Trial was pruned at iteration 2.


Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:03:14,544] Trial 99 pruned. Trial was pruned at iteration 0.


Number of finished trials: 100
Best trial: {'learning_rate': 0.10488972496870035, 'num_leaves': 100, 'max_depth': 10, 'min_child_samples': 1100, 'max_bin': 285, 'reg_alpha': 15, 'reg_lambda': 10, 'min_split_gain': 2.916286562179216, 'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.2, 'path_smooth': 1.0}


In [10]:
study.trials_dataframe().to_csv("/workspace/ligthgbm_trials3.csv",index=None)

In [11]:
study.optimize(func, n_trials=200,callbacks=[mlflow_callback])

Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:09:35,363] Trial 100 pruned. Trial was pruned at iteration 5.
[I 2023-01-13 11:09:38,110] Trial 101 pruned. Trial was pruned at iteration 0.


Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:09:50,965] Trial 102 pruned. Trial was pruned at iteration 11.


Training until validation scores don't improve for 100 rounds


...

[I 2023-01-13 11:41:53,430] Trial 294 pruned. Trial was pruned at iteration 4.


Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:41:59,778] Trial 295 pruned. Trial was pruned at iteration 1.
[I 2023-01-13 11:42:05,163] Trial 296 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:42:10,614] Trial 297 pruned. Trial was pruned at iteration 0.
[I 2023-01-13 11:42:15,812] Trial 298 pruned. Trial was pruned at iteration 0.


Training until validation scores don't improve for 100 rounds


[I 2023-01-13 11:42:24,018] Trial 299 pruned. Trial was pruned at iteration 2.


In [12]:
study.trials_dataframe().to_csv("/workspace/ligthgbm_trials3.csv",index=None)